# 🚀 **Capstone Project Curso Desarrolador 10x de Instituto de Inteligencia Artificial - Entregable 1**

## 📄 Información del Proyecto  
**Estudiante:** Araceli Fradejas Muñoz  
**Curso:** Curso Desarrollador 10x – Instituto de Inteligencia Artificial  
**Fecha:** 21/04/2025  

---

## 📝 Descripción del Proyecto: Análisis Automatizado de Comentarios - KelceTS

### 🔎 Contexto  
**KelceTS S.L.** es una startup ficticia de venta de zapatillas online en toda Europa. Actualmente, el equipo de calidad revisa los comentarios recibidos en redes sociales o por email de los clientes de forma manual, lo que genera cuellos de botella y respuestas poco homogéneas.

Además no están coordinadoslos los distintos equipos para dar respuesta al cliente ni mejorar el servicio postventa de sus productos.

---

## 🎯 Objetivo Principal

Diseñar un agente de IA generativa que automatice el ciclo completo de gestión de comentarios, incluyendo:

- 🔍 **Análisis estructurado de los comentarios** según criterios definidos (envío, embalaje, talla, calidad, expectativas…)
- 💬 **Generación de respuestas personalizadas** para clientes en 24 idiomas oficiales de la UE
- 📋 **Notificaciones automáticas** a los equipos internos de calidad y logística
- 📧 **Correos formales a proveedores externos**, siguiendo reglas internas de actuación
- 🧾 **Exportación de resultados** en formato Excel listo para auditoría

---

## 🛠️ Pasos de Implementación

1. Leer y preprocesar comentarios multilingües desde archivo `.txt`
2. Generar prompts estructurados a partir de reglas internas (archivos `.xlsx`)
3. Llamar a modelos de IA (OpenAI y Gemini) con fallback dinámico
4. Aplicar funciones de evaluación, clasificación y generación de comunicaciones
5. Traducir automáticamente los correos generados al idioma original del cliente
6. Separar errores y resultados válidos en dataframes estructurados
7. Exportar el resultado completo en un Excel multilingüe con pestañas organizadas

---

## ⚙️ Tecnologías Utilizadas

- **Python:** lenguaje principal para todo el procesamiento
- **OpenAI (gpt-4):** generación principal de análisis y comunicaciones
- **Gemini Pro (Google Cloud):** agente de respaldo (fallback) para asegurar robustez
- **Pandas:** manipulación avanzada de datos estructurados
- **LangChain + prompting estructurado:** para control y trazabilidad
- **Google Colab:** entorno reproducible en la nube
- **XlsxWriter:** exportación profesional a Excel con varias hojas
- **Gradio + Streamlit (futuros pasos):** interfaces amigables para usuarios finales

---

## 💡 Impacto Esperado

Este proyecto permite automatizar de forma fiable y escalable la evaluación de comentarios de clientes, proporcionando:

- ⏱️ Ahorro de tiempo en tareas repetitivas de atención al cliente  
- 📈 Mejora en la consistencia de las comunicaciones internas y externas  
- 🌍 Inclusión de clientes en múltiples idiomas sin depender de traductores humanos  
- 📊 Informes estructurados que permiten tomar decisiones con base en datos  



La integración de OpenAI y Gemini como doble motor garantiza continuidad incluso ante fallos, límites de uso o costes inesperados en uno de los servicios.


## 1. ⚙️ **Preparación del entorno**

En este apartado, instalamos e importamos las librerías necesarias y clonamos el repositorio público de GitHub que contiene los archivos del proyecto.

Además, conectamos el notebook con los datos disponibles en el repositorio de GitHub, dese su directorio `data` sin depender de Google Drive (en versiones anteriores realizamos esta dependencia y aqúi está optimizado para que cualquier usuario pueda ejecutarlo).

Los archivos disponibles incluyen:

- `BD Comentarios KelceTS.txt`
- `Reglas de como valorar un comentario KelceTS SL.xlsx`
- `Reglas de calidad clientes KelceTS SL.xlsx`
- `Reglas de comunicaciones equipos calidad y logística KelceTS SL.xlsx`
- `Reglas de medidas de calidad KelceTS SL.xlsx`



 🚧 En las siguientes celdas se instalan todas las dependencias necesarias y se configura el entorno base para comenzar con el análisis automatizado.

 >⚠️ Ejecuta esta celda antes de continuar con la carga de datos o modelos IA.

In [ ]:
#  1. Instalación de librerías necesarias
!pip install openai pandas matplotlib seaborn plotly openpyxl --quiet
!pip install -q google-generativeai python-dotenv
!pip install -U kaleido --quiet
!pip install xlsxwriter --quiet

#  2. Importación de librerías
import os
import json
import re
import time
import random
import pandas as pd
import xlsxwriter

# 3. Visualización
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

# 4. Modelos IA
import openai
import google.generativeai as genai

# 5. Estilo visual y reproducibilidad
plt.style.use("ggplot")
sns.set(style="whitegrid")
random.seed(42)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.4/169.4 kB 9.2 MB/s eta 0:00:00


### 1.1 ***🔐 Cargar claves de OpenAI y Gemini desde `.env` o desde `Secrets` en Colab*** **texto en negrita**

Este notebook puede ejecutarse de forma segura y privada gracias a un sistema mixto de carga de claves:

1. Primero busca un archivo `.env` en la raíz del proyecto, donde deben definirse las variables:
   - `OPENAI_API_KEY`
   - `GEMINI_API_KEY`

2. Si el archivo `.env` no está disponible, intentará cargar las claves desde **Google Colab Secrets** (`userdata.get()`).

Este enfoque garantiza seguridad (no se exponen las claves) y compatibilidad al compartir públicamente el notebook en GitHub.


⚠️ **MUY IMPORTANTE:**  

>Si estás ejecutando este notebook por primera vez, asegúrate de cargar las claves de API necesarias en el archivo `.env`.  
🔐 Si no lo has hecho aún, por favor, sigue las instrucciones para configurarlo antes de proceder.  
✅ Este paso es esencial para que el acceso a las APIs de **OpenAI** y **Gemini** funcione correctamente.  
❌ Si no se ha cargado el archivo `.env`, las claves no estarán disponibles, lo que generará un error en las celdas posteriores.  
🔑 Si lo prefieres, puedes guardar tus claves de **OpenAI** y **Gemini** como secretos pulsando el icono de la llave en este Colab.





In [ ]:
# Configuración de claves OpenAI + Gemini (.env o Colab Secrets)

import os
from dotenv import load_dotenv

# 1. Intentamos cargar desde archivo .env si existe
if os.path.exists(".env"):
    load_dotenv()
    print("📄 Archivo .env cargado.")
else:
    print("⚠️ No se encontró archivo .env. Intentando con Colab Secrets...")
    try:
        from google.colab import userdata
        os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
        os.environ["GEMINI_API_KEY"] = userdata.get("GEMINI_API_KEY")
        print("🔐 Claves cargadas desde Colab secrets.")
    except Exception as e:
        print(f"❌ No se encontraron claves: {str(e)}")

# 2. Configuramos clientes (¡aquí va la línea de OpenAI moderna!)
from openai import OpenAI
import google.generativeai as genai

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
genai.configure(api_key=os.getenv("GEMINI_API_KEY"))

# 3. Validación
if not os.getenv("OPENAI_API_KEY") or not os.getenv("GEMINI_API_KEY"):
    raise ValueError("❌ No se detectaron claves API.")
else:
    print("✅ Claves configuradas correctamente.")




⚠️ No se encontró archivo .env. Intentando con Colab Secrets...
🔐 Claves cargadas desde Colab secrets.
✅ Claves configuradas correctamente.


### ***1.3 🔄 Validación y traducción automática***

Ya que poder enviar comunicaciones en el idioma de los clientes va a ser una tarea de eficiencia muy importante que hay que cubrir para evitar errores relacionados con la caché y simplificar el flujo de traducción y validación, centralizamos todo el proceso en un único bloque de funciones.

Estas funciones permiten:
- Limpiar la caché y la memoria antes de cada nuevo comentario (`clear_cache`)
- Traducir automáticamente con fallback a OpenAI (`translate_comment`)
- Validar que la traducción tiene sentido y no ha perdido información clave (`validate_translation`)
- Procesar todo el flujo en un solo paso (`process_comment`)


In [ ]:
# 🔁 Utilidades para traducción y validación centralizadas

import gc
from functools import lru_cache
from openai import OpenAI

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# 1. Forzar recolección de basura y limpiar cache de funciones
def clear_cache():
    gc.collect()
    try:
        translate_comment.cache_clear()
        validate_translation.cache_clear()
    except NameError:
        pass

# 2. Función de traducción centralizada
@lru_cache(maxsize=128)
def translate_comment(comment: str, source_lang: str = None, target_lang: str = "es") -> str:
    prompt = f"Traduce este texto al {target_lang}:\n\n{comment}"
    if source_lang:
        prompt = f"El texto está en {source_lang}. " + prompt
    response = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "Actúa como traductor fiel."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.0,
    )
    return response.choices[0].message.content.strip()

# 3. Validación de traducción
@lru_cache(maxsize=128)
def validate_translation(original: str, translated: str) -> bool:
    orig_len = len(original.split())
    trans_len = len(translated.split())
    if trans_len < 0.7 * orig_len or trans_len > 1.3 * orig_len:
        return False
    for token in original.split():
        if token.istitle() and token.lower() not in translated.lower():
            return False
    return True

# 4. Función única para procesar un comentario completo
def process_comment(comment: str, source_lang: str = None):
    clear_cache()
    translation = translate_comment(comment, source_lang)
    is_valid = validate_translation(comment, translation)
    return {
        "original": comment,
        "translated": translation,
        "translation_valid": is_valid
    }


### ***1.4 📁 Clonación del repositorio de GitHub***

Clonamos el repositorio público donde se encuentran los archivos del proyecto. Se almacenan automáticamente en la carpeta `data/`.


In [ ]:
# Clonamos el repositorio público de GitHub con los archivos del proyecto
!git clone https://github.com/AraceliFradejas/Capstone-Project---Desarrollador10X-IIA---Araceli-Fradejas.git

# Definimos la ruta base donde están los datos dentro de la carpeta 'data'
ruta_base = "/content/Capstone-Project---Desarrollador10X-IIA---Araceli-Fradejas/data"
archivo_comentarios = f"{ruta_base}/BD Comentarios KelceTS.txt"

# Verificamos que los archivos están disponibles
print("📁 Archivos encontrados en la carpeta 'data':")
!ls -1 {ruta_base}



Cloning into 'Capstone-Project---Desarrollador10X-IIA---Araceli-Fradejas'...
remote: Enumerating objects: 84, done.
remote: Counting objects: 100% (84/84), done.
remote: Compressing objects: 100% (54/54), done.
remote: Total 84 (delta 46), reused 44 (delta 25), pack-reused 0 (from 0)
Receiving objects: 100% (84/84), 1.42 MiB | 12.00 MiB/s, done.
Resolving deltas: 100% (46/46), done.
📁 Archivos encontrados en la carpeta 'data':
'BD Comentarios KelceTS.txt'
'Kelce TS LOGO.png'
'Reglas de calidad clientes KelceTS SL.xlsx'
'Reglas de como valorar un comentario KelceTS SL.xlsx'
'Reglas de comunicaciones equipos calidad y logistica KelceTS SL.xlsx'
'Reglas de medidas de calidad KelceTS SL.xlsx'


## **2. Lectura de comentarios del archivo**

Creamos una función para extraer los comentarios desde el fichero `.txt`, identificando automáticamente el idioma y separando comentarios aunque estén en diferentes formatos.


In [ ]:
def leer_comentarios(archivo_path, limite=None):
    """
    Lee comentarios de un archivo de texto plano.
    Parámetros:
        archivo_path (str): Ruta del archivo con los comentarios
        limite (int): Límite de comentarios a leer (None para todos)
    """
    try:
        with open(archivo_path, 'r', encoding='utf-8') as f:
            contenido = f.read()

        # Posibles patrones multilingües para dividir los comentarios
        patrones = [
            r'(?:Comentario|Kommentar|Commentaire|Commento|Opmerking|Komentarz|Comentário|Kommentti|Σχόλιο)\\s+\\d+\\s*:\\s*(.*?)(?=(?:Comentario|Kommentar|Commentaire|Commento|Opmerking|Komentarz|Comentário|Kommentti|Σχόλιο)\\s+\\d+\\s*:|$)',
            r'(?<=\\n)\\d+\\s*(.*?)(?=\\n\\d+\\s*|$)'
        ]

        comentarios = []
        for patron in patrones:
            matches = re.findall(patron, contenido, re.DOTALL)
            if matches:
                comentarios = [match.strip() for match in matches if match.strip()]
                break

        if not comentarios:
            comentarios = [linea.strip() for linea in contenido.split('\n') if linea.strip()]

        if limite and len(comentarios) > limite:
            comentarios = comentarios[:limite]

        print(f"✅ Se han cargado {len(comentarios)} comentarios desde {archivo_path}")
        for i, comentario in enumerate(comentarios[:3]):
            print(f"Comentario {i+1} (muestra): {comentario[:100]}...")

        return comentarios

    except Exception as e:
        print(f"❌ Error al leer el archivo {archivo_path}: {str(e)}")
        return []


### 2.1 Validación de la lectura de comentarios

Se muestra un resumen del número total de comentarios cargados correctamente y una muestra aleatoria de 3 comentarios para verificar que se han leído con éxito desde el archivo `.txt`.


In [ ]:
# Leer comentarios desde el archivo usando la función definida
comentarios = leer_comentarios(archivo_comentarios)

# Validación visual
print(f"\n📊 Total de comentarios cargados: {len(comentarios)}")

# Mostrar una muestra aleatoria
if comentarios:
    print("\n🧾 Muestra de 3 comentarios:")
    for i, comentario in enumerate(random.sample(comentarios, min(3, len(comentarios))), 1):
        print(f"{i}. {comentario[:150]}...")
else:
    print("⚠️ No se encontraron comentarios.")


✅ Se han cargado 50 comentarios desde /content/Capstone-Project---Desarrollador10X-IIA---Araceli-Fradejas/data/BD Comentarios KelceTS.txt
Comentario 1 (muestra): Comentario 1: Estoy muy contento con mis nuevas zapatillas. Llegaron en menos de 48 horas, bien emba...
Comentario 2 (muestra): Kommentar 2: Als die Schuhe angekommen sind habe ich mich erstmal sehr gefreut und hätte wahrscheinl...
Comentario 3 (muestra): Σχόλιο 3: Έλαβα τα παπούτσια μου σε λιγότερο από 48 ώρες, τέλεια συσκευασμένα. Το μέγεθος είναι ακρι...

📊 Total de comentarios cargados: 50

🧾 Muestra de 3 comentarios:
1. Komentar 41: Primio sam svoje cipele u manje od 48 sati, savršeno zapakirane. Veličina je upravo odgovarajuća, a materijali su izvrsne kvalitete. Kori...
2. Kommentti 8: Sain kengät alle 48 tunnissa, täydellisesti pakattuina. Koko on juuri oikea ja materiaalit ovat erinomaista laatua. Käytän niitä päivittä...
3. Kommentar 2: Als die Schuhe angekommen sind habe ich mich erstmal sehr gefreut und hätte wahrsc

### 2.2 Cargar los comentarios en una variable global

Utilizamos la función definida anteriormente para leer los comentarios desde el archivo `.txt` y los almacenamos en una variable global llamada `comentarios`.


In [ ]:
print("\n--- Cargando comentarios del archivo ---")
comentarios = leer_comentarios(archivo_comentarios)

# Guardamos la lista en una variable accesible globalmente
import sys
sys.modules['__main__'].comentarios = comentarios

print(f"\n✅ Variable 'comentarios' creada con {len(comentarios)} comentarios")



--- Cargando comentarios del archivo ---
✅ Se han cargado 50 comentarios desde /content/Capstone-Project---Desarrollador10X-IIA---Araceli-Fradejas/data/BD Comentarios KelceTS.txt
Comentario 1 (muestra): Comentario 1: Estoy muy contento con mis nuevas zapatillas. Llegaron en menos de 48 horas, bien emba...
Comentario 2 (muestra): Kommentar 2: Als die Schuhe angekommen sind habe ich mich erstmal sehr gefreut und hätte wahrscheinl...
Comentario 3 (muestra): Σχόλιο 3: Έλαβα τα παπούτσια μου σε λιγότερο από 48 ώρες, τέλεια συσκευασμένα. Το μέγεθος είναι ακρι...

✅ Variable 'comentarios' creada con 50 comentarios


### 2.3 Verificar carpeta y archivo de comentarios

Antes de avanzar, comprobamos si la ruta del archivo de comentarios existe y si el archivo está correctamente guardado en esa ubicación. Mostramos también los primeros comentarios para validar que el archivo se puede leer.


In [ ]:
print(f"\n📂 Ruta de trabajo: {ruta_base}")

if os.path.exists(ruta_base):
    print(f"✅ Carpeta encontrada correctamente")
    print("Archivos disponibles en la carpeta:")
    for archivo in os.listdir(ruta_base):
        print(f"  • {archivo}")

    if os.path.exists(archivo_comentarios):
        print(f"\n✅ Archivo de comentarios encontrado: {archivo_comentarios}")
        with open(archivo_comentarios, 'r', encoding='utf-8') as f:
            primeras_lineas = ''.join(f.readlines()[:5])
            print(f"Primeras líneas del archivo:\n{primeras_lineas}")
    else:
        print(f"\n❌ No se encontró el archivo de comentarios: {archivo_comentarios}")
else:
    print(f"❌ Ruta no encontrada: {ruta_base}")



📂 Ruta de trabajo: /content/Capstone-Project---Desarrollador10X-IIA---Araceli-Fradejas/data
✅ Carpeta encontrada correctamente
Archivos disponibles en la carpeta:
  • Reglas de comunicaciones equipos calidad y logistica KelceTS SL.xlsx
  • Reglas de como valorar un comentario KelceTS SL.xlsx
  • BD Comentarios KelceTS.txt
  • .gitkeep
  • Reglas de medidas de calidad KelceTS SL.xlsx
  • Kelce TS LOGO.png
  • Reglas de calidad clientes KelceTS SL.xlsx

✅ Archivo de comentarios encontrado: /content/Capstone-Project---Desarrollador10X-IIA---Araceli-Fradejas/data/BD Comentarios KelceTS.txt
Primeras líneas del archivo:
Comentario 1: Estoy muy contento con mis nuevas zapatillas. Llegaron en menos de 48 horas, bien embaladas, la talla es perfecta y los materiales son de buena calidad. Las uso para correr todos los días y han superado mis expectativas.
Kommentar 2: Als die Schuhe angekommen sind habe ich mich erstmal sehr gefreut und hätte wahrscheinlich 5 Sterne vergeben! Sie haben gut gepas

### 2.4  Comentarios cargados correctamente

Mostramos una muestra representativa de los primeros comentarios procesados, para verificar que la lectura y segmentación se han realizado correctamente.


In [ ]:
# Mostramos un resumen rápido
print(f"🔎 Total de comentarios cargados: {len(comentarios)}")

# Mostramos una muestra aleatoria de 3 comentarios
print("\n🧾 Muestra de comentarios:")
for i, comentario in enumerate(random.sample(comentarios, 3), 1):
    print(f"{i}. {comentario[:150]}...")


🔎 Total de comentarios cargados: 50

🧾 Muestra de comentarios:
1. Kommentar 48: Leveringen var inden for de lovede 96 timer, og emballagen var intakt. Størrelsen er korrekt, men materialet er ikke af særlig god kvali...
2. Kommentár 18: A cipő több mint egy hét alatt érkezett meg, és amikor kinyitottam a dobozt, az egyik sarokban sérült volt. Szerencsére a termék sértetl...
3. Comentario 16: Son un poco plasticosas por fuera. Son bonitas. Me quedan algo grandes, el 46, quizás sea el problema de usar 45 y medio. Con plantilla...


## **3. Cargar reglas de valoración desde archivos Excel**

En este paso cargamos las reglas de negocio definidas por KelceTS desde los archivos `.xlsx` que se encuentran en la carpeta `data`.

Estas reglas incluyen:

- ✅ Reglas de valoración de comentarios
- ✅ Reglas de calidad para clientes
- ✅ Reglas de comunicaciones internas (calidad y logística)
- ✅ Reglas de medidas de calidad a aplicar

La función se encarga de leer cada archivo Excel, transformarlo en diccionarios y dejarlos listos para el análisis posterior.



In [ ]:
def cargar_reglas_excel(ruta_base):
    """
    Carga todas las reglas de negocio desde archivos Excel ubicados en la carpeta 'data'.
    Devuelve un diccionario con claves: valoracion, calidad_clientes, comunicaciones_equipos, medidas_calidad.
    """
    archivos = {
        "valoracion": "Reglas de como valorar un comentario KelceTS SL.xlsx",
        "calidad_clientes": "Reglas de calidad clientes KelceTS SL.xlsx",
        "comunicaciones_equipos": "Reglas de comunicaciones equipos calidad y logistica KelceTS SL.xlsx",
        "medidas_calidad": "Reglas de medidas de calidad KelceTS SL.xlsx"
    }

    reglas = {}

    for clave, nombre_archivo in archivos.items():
        ruta_completa = os.path.join(ruta_base, nombre_archivo)

        try:
            if os.path.exists(ruta_completa):
                df = pd.read_excel(ruta_completa)
                reglas[clave] = df.to_dict("records")
                print(f"✅ {clave.upper()} cargadas correctamente ({len(df)} reglas)")
            else:
                print(f"❌ Archivo no encontrado: {nombre_archivo}")
                reglas[clave] = []
        except Exception as e:
            print(f"❌ Error al cargar {nombre_archivo}: {str(e)}")
            reglas[clave] = []

    return reglas


In [ ]:
# Ejecutamos la carga de reglas desde la carpeta /data del repositorio
reglas = cargar_reglas_excel(ruta_base)

✅ VALORACION cargadas correctamente (7 reglas)
✅ CALIDAD_CLIENTES cargadas correctamente (4 reglas)
✅ COMUNICACIONES_EQUIPOS cargadas correctamente (24 reglas)
✅ MEDIDAS_CALIDAD cargadas correctamente (5 reglas)


### 3.1 Verificación de reglas cargadas:
Mostramos cuántas reglas se han detectado por cada categoría y un pequeño ejemplo de su contenido.

In [ ]:
for nombre, datos in reglas.items():
    print(f"\n📊 Reglas de {nombre}: {len(datos)} registros")
    if datos:
        ejemplo = {k: v for k, v in list(datos[0].items())[:3]}
        print(f"Ejemplo: {ejemplo}")



📊 Reglas de valoracion: 7 registros
Ejemplo: {'Pregunta para detrminar la valoración ': '<Identificación del idioma del comentario >', 'Variable para establecer la valoración de respuesta': 'Idioma comentario cliente', 'Detalle respuesta positiva': 'Si el idioma es castellano , debemos utilizar castellano en la respuesta al cliente '}

📊 Reglas de calidad_clientes: 4 registros
Ejemplo: {'Valoración feedback del cliente': 'Calidad del envío', 'Valoración Positiva': 'Cumple que el envío se ha recibido en menos de 96h o no se ha detectado ninguna anomalía en el embalaje', 'Valoración Negativa': 'No cumple que el envío se ha recibido en menos de 96h o se ha detectado alguna anomalía en el embalaje'}

📊 Reglas de comunicaciones_equipos: 24 registros
Ejemplo: {'Acciones a realizar en equipos de la startup': 'Calidad del envío: envío retrasado', 'Valoración Positiva': 'Cumple que el envío se ha recibido en menos de 96h', 'Valoración Negativa': 'No cumple que el envío se ha recibido en menos 

## **4. Ejecutar análisis con agente de IA**

Creamos una función `analizar_comentario()` que procesa cada comentario de cliente utilizando la API de OpenAI y las reglas definidas. Para cada comentario, realizamos:

1. 📍 Identificación del idioma
2. 📚 Resumen del contenido y extracción de factores clave
3. 😊 Análisis de sentimiento
4. 📊 Valoración con base en reglas
5. 💬 Generación de respuesta al cliente
6. 🏷️ Notificaciones internas (si aplica)
7. ✉️ Correo para proveedor (si aplica)

Se devuelve un diccionario estructurado con todos los resultados.



In [ ]:
def generar_prompt_con_reglas(reglas, comentario, traduccion=None):
    """
    Genera un prompt estructurado para el agente IA según las reglas internas.
    """

    prompt_base = """
    Eres un asistente experto para el equipo de calidad de KelceTS S.L., una startup de zapatillas.
    Analiza el siguiente comentario de un cliente siguiendo EXACTAMENTE las reglas proporcionadas.

    COMENTARIO DEL CLIENTE:
    {comentario}

    INSTRUCCIONES DE ANÁLISIS (responde a cada punto):
    1. Identifica el idioma exacto del comentario.
    2. ¿Las zapatillas se recibieron en menos de 96h? (sí/no/no mencionado)
    3. ¿El embalaje estaba dañado? (sí/no/no mencionado)
    4. ¿La talla es correcta? (sí/no/no mencionado)
    5. ¿Los materiales son de buena calidad? (sí/no/parcialmente/no mencionado)
    6. ¿Qué tipo de uso hace el cliente? (diario/ocasional/no mencionado)
    7. ¿El producto cumple las expectativas? (sí/no/parcialmente)

    REGLAS PARA VALORACIONES NEGATIVAS:
    - Si menciona problemas con materiales, valora como "no" en calidad de materiales.
    - Si menciona que la talla es grande o pequeña, valora como "no" en talla correcta.
    - Si menciona retraso en la entrega (más de 96h), valora como "no" en envío en 96h.
    - Si hay al menos una valoración negativa, el producto no cumple expectativas.

    REGLAS PARA COMUNICACIONES:
    - Email al cliente:
        - En el idioma original del cliente
        - Tono cercano y uso del "tú"
        - Incluir soluciones específicas según el problema
        - Firmar como "KelceTS Team"
    - Notificación interna y email a proveedor:
        - En español
        - Claros y accionables
        - Firmar con el rol correspondiente

    FORMATO DE RESPUESTA:
    Devuelve solo un JSON con esta estructura (usa comillas dobles y no añadas explicaciones):

    {{
      "analisis": {{
        "idioma": "...",
        "envio_96h": "...",
        "embalaje_danado": "...",
        "talla_correcta": "...",
        "materiales_calidad": "...",
        "tipo_uso": "...",
        "cumple_expectativas": "..."
      }},
      "valoracion": "...",
      "comunicaciones": {{
        "email_cliente": "...",
        "email_cliente_traduccion": "...",
        "notificacion_interna": "...",
        "email_proveedor": "..."
      }}
    }}
    """

    return prompt_base.format(comentario=comentario)


### 4.1 🧠 Análisis de Comentario con Agente de IA

Esta función utiliza la API de OpenAI para analizar cada comentario individual según las reglas internas de KelceTS S.L.:
- Evalúa idioma, calidad, embalaje, talla, materiales, etc.
- Clasifica automáticamente la valoración como positiva, negativa o neutra.
- Genera comunicaciones internas y externas.
- Extrae el bloque JSON de la respuesta de manera robusta con expresiones regulares, incluso si el modelo devuelve texto adicional.


#### 4.1.1 🛡️ Protección frente a errores JSON

A veces el modelo devuelve respuestas que no contienen JSON válido (vacías, incompletas o malformadas), lo cual genera un error `JSONDecodeError` al intentar convertir la cadena en un diccionario con `json.loads()`.

Para evitar que se interrumpa la ejecución del análisis de todos los comentarios, hemos añadido un bloque `try/except` dentro de la función `analizar_comentario()` que:
- Detecta si el bloque JSON está malformado,
- Captura el error de decodificación,
- Y devuelve un diccionario con el comentario original y el error encontrado.

Esto nos permite registrar qué comentarios han fallado, continuar con los que sí funcionan, y mostrar la vista previa de errores al final.


In [ ]:
def analizar_comentario(comentario, reglas):
    """
    Usa un agente de IA para analizar un comentario según las reglas internas.
    Devuelve un diccionario con idioma, análisis, valoración, comunicaciones, etc.
    """

    try:
        prompt_sistema = {
            "role": "system",
            "content": "Eres un asistente especializado en KelceTS S.L. que analiza comentarios de clientes y genera respuestas siguiendo reglas internas."
        }

        prompt_usuario = {
            "role": "user",
            "content": generar_prompt_con_reglas(reglas, comentario)
        }

        respuesta = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[prompt_sistema, prompt_usuario],
            temperature=0.4
        )

        texto = respuesta.choices[0].message["content"]

        # 🧠 Buscar el bloque JSON dentro del texto
        match = re.search(r"\{[\s\S]*\}", texto)
        if match:
            json_text = match.group()

            # ✅ Protección frente a errores de parseo JSON
            try:
                resultado_dict = json.loads(json_text)
                resultado_dict["comentario_original"] = comentario
                return resultado_dict
            except json.JSONDecodeError as e:
                return {
                    "comentario_original": comentario,
                    "error": f"JSONDecodeError: {str(e)}",
                    "respuesta_raw": texto
                }
        else:
            return {
                "comentario_original": comentario,
                "error": "❌ No se encontró JSON válido en la respuesta.",
                "respuesta_raw": texto
            }

    except Exception as e:
        return {
            "comentario_original": comentario,
            "error": f"❌ Error en la llamada a OpenAI: {str(e)}"
        }


### 4.2 Validación de comunicaciones multiidioma

Definimos una función especializada que valida y corrige las comunicaciones generadas:
- Asegura que los emails al cliente estén en el idioma original del cliente
- Verifica que contengan todos los elementos obligatorios (email de contacto, firma correcta)
- Añade las medidas de calidad correspondientes según el tipo de problema
- Genera automáticamente traducciones y correcciones cuando sea necesario
- Soporta todos los idiomas de la Unión Europea

## **5. Ejecutar análisis masivo y aplicar comunicaciones oficiales**

En este paso recorremos todos los comentarios cargados y aplicamos el flujo completo de análisis y corrección:

1. 🧠 Procesamos cada comentario con `analizar_comentario()` o `analizar_con_fallback()`, que aplica las reglas internas mediante IA.
2. 🛠️ Aplicamos las reglas oficiales con `aplicar_comunicaciones_oficiales()` para asegurarnos de que:
   - Las respuestas al cliente siguen exactamente lo definido en los Excels del ejercicio.
   - Se incluyen las medidas correctivas (descuento 5%, 25%, cambio de talla...) según las incidencias detectadas.
   - Se generan correctamente los textos para cliente, proveedor y departamentos internos, con el tono, idioma y firma adecuados.
3. 💾 Guardamos todos los resultados en una lista estructurada (`resultados`), lista para visualizar, exportar o analizar con gráficos.

Este paso garantiza que todas las comunicaciones cumplen al 100% con el Prompt definido.



### 5.1 🧪 Verificación inicial de análisis con IA (antes del procesamiento masivo)

Antes de analizar todos los comentarios, hacemos una prueba individual para asegurarnos de que la función `analizar_comentario_fallback()` funciona correctamente y devuelve los campos esperados (`analisis`, `valoracion`, `comunicaciones`).

Esto nos permite detectar errores tempranos (claves mal cargadas, estructura del prompt incorrecta, errores de conexión, etc.) sin tener que procesar los 50 comentarios.


#### 5.1.1 🧠 *Función de análisis con Fallback (OpenAI → Gemini)*

Esta función `analizar_con_fallback()` realiza el análisis del comentario usando primero el modelo `gpt-3.5-turbo` de OpenAI. Si la llamada falla (por error de clave, límite de uso, etc.), automáticamente utiliza el modelo `gemini-pro` de Google AI como respaldo.

Este enfoque garantiza robustez, evitando que el análisis completo se detenga por un solo fallo de conexión o error de proveedor.

- ✅ Si OpenAI responde bien → usa su resultado.
- 🔁 Si OpenAI falla → cambia a Gemini automáticamente.



##### 5.1.1.1. 🔁 *Uso de fallback entre OpenAI y Gemini*

En esta función usamos un mecanismo de respaldo: intentamos generar la respuesta con OpenAI, y si falla (por timeout, límite, etc.), se lanza una segunda llamada con Gemini. La función no convierte la respuesta a JSON, solo devuelve el texto crudo generado por el modelo.

La protección frente a errores JSON la añadimos en la celda que utiliza esta función (`json.loads()` o `extraer_json_desde_texto()`), y no aquí dentro.



In [ ]:
def analizar_con_fallback(comentario, prompt_openai, prompt_gemini):
    try:
        # 🧠 Primero intentamos con OpenAI
        respuesta = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "Eres un asistente experto de KelceTS S.L."},
                {"role": "user", "content": prompt_openai}
            ],
            temperature=0.4,
        )
        return respuesta.choices[0].message.content

    except Exception as e:
        print(f"⚠️ Error con OpenAI: {e}")
        print("🔁 Usando Gemini como respaldo...")

        try:
            model = genai.GenerativeModel("gemini-pro")
            respuesta = model.generate_content(prompt_gemini)
            return respuesta.text
        except Exception as err:
            print(f"❌ Error con Gemini: {err}")
            return {
                "comentario_original": comentario,
                "error": str(err)
            }


In [ ]:
# Probaremos el primer comentario antes del análisis masivo
comentario_test = comentarios[0]

# Generamos el prompt usando las reglas
prompt_test = generar_prompt_con_reglas(reglas, comentario_test)

# Ejecutamos el análisis con fallback OpenAI → Gemini
respuesta_test = analizar_con_fallback(comentario_test, prompt_test, prompt_test)

# Mostramos resultado crudo
print("🔍 Respuesta de la IA:")
print(respuesta_test)


🔍 Respuesta de la IA:
{
  "analisis": {
    "idioma": "Español",
    "envio_96h": "Sí",
    "embalaje_danado": "No",
    "talla_correcta": "Sí",
    "materiales_calidad": "Sí",
    "tipo_uso": "Diario",
    "cumple_expectativas": "Sí"
  },
  "valoracion": "Positiva",
  "comunicaciones": {
    "email_cliente": "Estimado/a cliente, nos alegra saber que estás contento/a con tus nuevas zapatillas. Si en el futuro necesitas algo más, no dudes en contactarnos. ¡Gracias por confiar en nosotros!",
    "email_cliente_traduccion": "Dear customer, we are glad to hear that you are happy with your new sneakers. If you need anything in the future, feel free to reach out to us. Thank you for trusting us!",
    "notificacion_interna": "El cliente ha expresado su satisfacción con las zapatillas recibidas. Valoración positiva en todos los aspectos evaluados.",
    "email_proveedor": "Estimado proveedor, queremos felicitarte por la eficacia en el envío de las zapatillas, así como por la calidad de los ma

#### 5.1.1.2 🧼 *Limpieza y normalización de la respuesta del agente IA*

Una vez obtenida la respuesta del agente de IA, es importante asegurarnos de que:

- Los valores están correctamente formateados (todo en minúsculas).
- No hay traducción innecesaria al español si el comentario original ya está en español.
- La estructura del JSON es consistente con lo esperado en el flujo de análisis.

Esta función garantiza que los datos se puedan usar sin errores en el análisis masivo, visualización y exportación posterior.


In [ ]:
def normalizar_resultado_ia(resultado):
    """
    Limpia y normaliza la estructura y valores del JSON generado por la IA.
    Convierte campos a minúsculas, asegura estructura y elimina traducciones innecesarias.
    """
    if "analisis" in resultado:
        for k, v in resultado["analisis"].items():
            if isinstance(v, str):
                resultado["analisis"][k] = v.strip().lower()

    if "valoracion" in resultado and isinstance(resultado["valoracion"], str):
        resultado["valoracion"] = resultado["valoracion"].strip().lower()

    if "comunicaciones" in resultado:
        idioma = resultado.get("analisis", {}).get("idioma", "").lower()
        if idioma == "español":
            resultado["comunicaciones"]["email_cliente_traduccion"] = ""  # Vaciar si no aplica

    return resultado


In [ ]:
# Normalizamos el resultado obtenido de la IA (respuesta_test)
resultado_normalizado = normalizar_resultado_ia(json.loads(respuesta_test))

# Mostramos el resultado limpio
print("🧼 Resultado normalizado:")
print(json.dumps(resultado_normalizado, indent=2, ensure_ascii=False))


🧼 Resultado normalizado:
{
  "analisis": {
    "idioma": "español",
    "envio_96h": "sí",
    "embalaje_danado": "no",
    "talla_correcta": "sí",
    "materiales_calidad": "sí",
    "tipo_uso": "diario",
    "cumple_expectativas": "sí"
  },
  "valoracion": "positiva",
  "comunicaciones": {
    "email_cliente": "Estimado/a cliente, nos alegra saber que estás contento/a con tus nuevas zapatillas. Si en el futuro necesitas algo más, no dudes en contactarnos. ¡Gracias por confiar en nosotros!",
    "email_cliente_traduccion": "",
    "notificacion_interna": "El cliente ha expresado su satisfacción con las zapatillas recibidas. Valoración positiva en todos los aspectos evaluados.",
    "email_proveedor": "Estimado proveedor, queremos felicitarte por la eficacia en el envío de las zapatillas, así como por la calidad de los materiales utilizados. Seguimos recibiendo comentarios positivos de nuestros clientes. ¡Gracias por tu colaboración! - Equipo de Calidad KelceTS"
  }
}


#### 5.1.1.3 🧼 *Limpieza segura del JSON devuelto por la IA*

Los modelos generativos como OpenAI o Gemini pueden añadir texto extra antes o después del JSON esperado. Para evitar errores del tipo `JSONDecodeError`, esta función busca automáticamente el primer bloque JSON válido dentro del texto y lo extrae para analizarlo de forma segura.


In [ ]:
def extraer_json_desde_texto(texto):
    """
    Extrae el primer bloque JSON válido desde un texto generado por IA.
    Devuelve un diccionario o lanza excepción si no encuentra JSON.
    """
    try:
        # Buscar el primer bloque que parezca un JSON (delimitado por llaves)
        match = re.search(r"\{[\s\S]*\}", texto)
        if match:
            json_texto = match.group()
            return json.loads(json_texto)
        else:
            raise ValueError("❌ No se encontró JSON en la respuesta.")
    except Exception as e:
        raise ValueError(f"⚠️ Error al extraer JSON: {e}")


### 5.2 🧠 Análisis automático de todos los comentarios (con fallback OpenAI → Gemini)

En esta sección se ejecuta el análisis automatizado sobre todos los comentarios usando el agente IA. Se utiliza un sistema de fallback que prioriza OpenAI y recurre a Gemini si ocurre algún error.

El flujo para cada comentario es el siguiente:

1. Se genera el prompt y se analiza con `analizar_con_fallback()`.
2. Se normaliza el resultado con `normalizar_resultado_ia()`.
3. Se asocia el comentario original para trazabilidad.
4. Si el análisis falla, se captura el error sin detener el bucle.

Esto asegura una ejecución robusta, con resultados válidos y errores trazables en el `DataFrame` final.


#### 5.2.1 🧪 *Protección de json.loads() en el análisi principal*

Al usar la función `analizar_con_fallback()`, obtenemos una cadena de texto que se espera esté en formato JSON. Sin embargo, puede haber casos donde la respuesta esté vacía, incompleta o malformada, generando un `JSONDecodeError`.

Para evitar que se interrumpa el procesamiento del resto de comentarios, envolvemos `json.loads()` en un bloque `try/except`. Si el JSON no se puede parsear correctamente, registramos el error junto con el comentario original.


In [ ]:
def safe_json_parse(response_str):
    try:
        return json.loads(response_str)
    except json.JSONDecodeError as e:
        print(f"⚠️ Error al parsear JSON: {e}")
        return None

resultados = []

# Procesamos todos los comentarios uno por uno
for i, comentario in enumerate(comentarios, 1):
    print(f"\n🔍 Procesando comentario {i}/{len(comentarios)}...")

    try:
        # Generamos prompt y analizamos
        prompt = generar_prompt_con_reglas(reglas, comentario)
        respuesta = analizar_con_fallback(comentario, prompt, prompt)

        # Convertimos respuesta a diccionario si es JSON válido
        resultado = safe_json_parse(respuesta) if isinstance(respuesta, str) else respuesta

        if resultado:
            resultado = normalizar_resultado_ia(resultado)
            resultado["comentario_original"] = comentario
        else:
            resultado = {
                "comentario_original": comentario,
                "error": "❌ Respuesta vacía o malformada (no es JSON)"
            }

    except Exception as e:
        resultado = {
            "comentario_original": comentario,
            "error": repr(e)
        }
        print(f"❌ Error en comentario {i}: {e}")

    resultados.append(resultado)

print("\n✅ Todos los comentarios han sido procesados correctamente.")



🔍 Procesando comentario 1/50...

🔍 Procesando comentario 2/50...

🔍 Procesando comentario 3/50...

🔍 Procesando comentario 4/50...

🔍 Procesando comentario 5/50...

🔍 Procesando comentario 6/50...

🔍 Procesando comentario 7/50...

🔍 Procesando comentario 8/50...

🔍 Procesando comentario 9/50...

🔍 Procesando comentario 10/50...

🔍 Procesando comentario 11/50...

🔍 Procesando comentario 12/50...

🔍 Procesando comentario 13/50...

🔍 Procesando comentario 14/50...

🔍 Procesando comentario 15/50...

🔍 Procesando comentario 16/50...

🔍 Procesando comentario 17/50...

🔍 Procesando comentario 18/50...

🔍 Procesando comentario 19/50...

🔍 Procesando comentario 20/50...

🔍 Procesando comentario 21/50...

🔍 Procesando comentario 22/50...
⚠️ Error al parsear JSON: Expecting value: line 1 column 1 (char 0)

🔍 Procesando comentario 23/50...

🔍 Procesando comentario 24/50...

🔍 Procesando comentario 25/50...

🔍 Procesando comentario 26/50...

🔍 Procesando comentario 27/50...

🔍 Procesando comentari

### 5.3 Verificación final de reglas de calidad y comunicaciones (multiidioma)

Una vez procesados los comentarios, aplicamos una verificación final para garantizar que se han respetado correctamente todas las reglas de calidad de KelceTS, independientemente del idioma del comentario.

Esta verificación corrige:
- Valoraciones que deberían ser negativas pero fueron clasificadas mal.
- Traducciones innecesarias si el idioma ya es español.
- Campos de comunicaciones que falten, completándolos con mensajes por defecto.

Esta capa final asegura que el informe generado para la entrega cumpla con todos los criterios definidos en los documentos de reglas.


In [ ]:
def verificacion_final_reglas_kelcets_multiidioma(resultado, texto_original, idioma, reglas):
    """
    Realiza una última verificación para asegurar que todas las reglas de KelceTS
    se han aplicado correctamente, con soporte para todos los idiomas.
    """
    analisis = resultado.get("analisis", {})
    valoracion = resultado.get("valoracion", "")
    comunicaciones = resultado.get("comunicaciones", {})

    # 1. Corrige la valoración si hay problemas
    tiene_negativo = any(analisis.get(k) == "no" for k in ["envio_96h", "embalaje_danado", "talla_correcta", "materiales_calidad"])
    if tiene_negativo and valoracion != "negativa":
        resultado["valoracion"] = "negativa"

    # 2. Vacía traducción si idioma ya es español
    if idioma == "español":
        comunicaciones["email_cliente_traduccion"] = ""

    # 3. Reemplaza campos vacíos por texto por defecto
    comunicaciones.setdefault("notificacion_interna", "No se generó notificación interna.")
    comunicaciones.setdefault("email_proveedor", "No se generó email al proveedor.")
    comunicaciones.setdefault("email_cliente", "No se generó email al cliente.")
    comunicaciones.setdefault("email_cliente_traduccion", "")

    resultado["comunicaciones"] = comunicaciones
    return resultado

# ✅ Aplicamos la verificación final con control de errores
for resultado in resultados:
    if "analisis" in resultado and isinstance(resultado["analisis"], dict):
        idioma = resultado.get("analisis", {}).get("idioma", "español")
        texto_original = resultado.get("comentario_original", "")

        try:
            resultado = verificacion_final_reglas_kelcets_multiidioma(resultado, texto_original, idioma, reglas)
        except Exception as e:
            print(f"⚠️ Error verificando comentario: {texto_original[:100]}...\n{e}")


#### 5.3.1  *Traducción automática del mensaje al idioma original del cliente*

Si el idioma del comentario no es español, traducimos automáticamente el mensaje generado para el cliente utilizando Gemini. Si Gemini falla, se hace fallback con OpenAI (GPT-4). Así aseguramos que las medidas de calidad aplicadas lleguen al cliente en su idioma.


In [ ]:
def traducir_texto(texto, idioma_destino):
    clave = f"{idioma_destino.lower()}::{texto.strip()}"

    if clave in traducciones_cache:
        return traducciones_cache[clave]

    prompt_traduccion = f"Traduce el siguiente mensaje al idioma {idioma_destino} manteniendo el tono profesional, claro y empático:\n\n\"{texto}\""

    try:
        print(f"🌍 Traduciendo a {idioma_destino}...")
        time.sleep(3)  # Para evitar sobrecarga y cuelgues
        response = client.chat.completions.create(
            model="gpt-4",
            messages=[
                {"role": "system", "content": "Eres un traductor profesional de atención al cliente."},
                {"role": "user", "content": prompt_traduccion}
            ],
            temperature=0.2
        )
        traduccion = response.choices[0].message.content.strip()
    except Exception as e_openai:
        print(f"❌ Error con OpenAI: {e_openai}")
        traduccion = "⚠️ No se pudo traducir el mensaje automáticamente."

    traducciones_cache[clave] = traduccion
    return traduccion


### 5.4 🧠 Versión mejorada de `aplicar_comunicaciones_oficiales()`

Esta versión:

- Asegura que se incluyan **todas las medidas de calidad que aplican al comentario**.
- Copia automáticamente el `email_cliente` como traducción si el idioma no es español.
- Añade los textos correctos para equipos internos y proveedores.



In [ ]:
def aplicar_comunicaciones_oficiales(resultado):
    analisis = resultado.get("analisis", {})
    idioma = analisis.get("idioma", "español")
    valoracion = resultado.get("valoracion", "")

    firma = "\n\nUn cordial saludo,\nKelceTS Team\natencionalcliente@kelcetssl.com"

    piezas_cliente = []
    acciones_internas = []
    acciones_proveedor = []

    if valoracion == "positiva":
        mensaje_cliente = f"¡Gracias por confiar en KelceTS! Nos alegra saber que estás satisfecho con tu compra.{firma}"
        mensaje_interno = ""
        mensaje_proveedor = ""

    elif valoracion == "negativa":
        if analisis.get("envio_96h") == "no":
            piezas_cliente.append("Hemos detectado un retraso en la entrega. Te ofrecemos un 5% de descuento en tu próxima compra.")
            acciones_proveedor.append("- Contactar con el proveedor para mejorar el tiempo de entrega (menos de 24h).")

        if analisis.get("embalaje_danado") == "sí":
            piezas_cliente.append("Hemos registrado que el embalaje llegó dañado. Te ofrecemos un 5% de descuento por las molestias.")
            acciones_proveedor.append("- Evaluar calidad del embalaje con proveedor logístico.")

        if analisis.get("talla_correcta") == "no":
            piezas_cliente.append("Vamos a enviarte en menos de 72h un nuevo par con la talla correcta, sin coste. Por favor, prepara el par anterior para su recogida.")
            acciones_proveedor.append("- Envío urgente de nuevo par con talla correcta. Recojo del anterior.")

        if analisis.get("materiales_calidad") == "no":
            piezas_cliente.append("Te ofrecemos un 25% de descuento y envío gratis. Enviaremos un miembro del staff en 72h para recoger el producto defectuoso.")
            acciones_proveedor.append("- Revisar materiales con proveedor. Plan de sustitución urgente.")

        if piezas_cliente:
            mensaje_cliente = "Lamentamos mucho los inconvenientes encontrados en tu compra. " + " ".join(piezas_cliente) + firma
            mensaje_interno = (
                "Este cliente ha recibido una valoración negativa. Deben notificarse los siguientes puntos:\n"
                + "\n".join(acciones_proveedor) +
                "\n\nFirmado: Asistente IA de KelceTS S.L."
            )
            mensaje_proveedor = (
                "Estimado proveedor:\n"
                + "\n".join(acciones_proveedor) +
                "\n\nAtentamente,\nRodrigo Clemente, Director de Logística de KelceTS S.L."
            )
        else:
            mensaje_cliente = "Gracias por tu opinión. Estamos revisando los aspectos mencionados para mejorar nuestro servicio." + firma
            mensaje_interno = ""
            mensaje_proveedor = ""

    elif valoracion == "parcialmente":
        mensaje_cliente = f"Gracias por tu opinión. Vamos a revisar los aspectos que mencionas para seguir mejorando.{firma}"
        mensaje_interno = ""
        mensaje_proveedor = ""

    else:
        mensaje_cliente = "Gracias por tu comentario. Lo tendremos en cuenta para seguir mejorando nuestro servicio." + firma
        mensaje_interno = ""
        mensaje_proveedor = ""

    # Asignar textos finales
    resultado["email_cliente"] = mensaje_cliente
    resultado["email_cliente_traduccion"] = traducir_texto(mensaje_cliente, idioma) if idioma != "español" else ""
    resultado["notificacion_interna"] = mensaje_interno
    resultado["email_proveedor"] = mensaje_proveedor

    return resultado


#### 5.4.1*Verificación de comunicaciones generadas (vista previa)*

Mostramos en tabla los primeros comentarios procesados, con sus respectivos correos al cliente, notificaciones internas y correos al proveedor. Esta revisión sirve para validar que se aplican correctamente todas las medidas de calidad, descuentos y respuestas personalizadas.

##### 5.4.1.1 ✅ Conversión de resultados en DataFrame y aplanamiento de campos anidados

En este bloque:

1. Convertimos la lista `resultados` en dos DataFrames: `df_validos` y `df_errores`.
2. Aplanamos el campo `analisis` (que contiene idioma, valoración, etc.).
3. Aplanamos el campo `comunicaciones` (que contiene los textos de emails generados).
4. Mostramos las columnas clave para verificar que todo se ha generado correctamente.



In [ ]:
#  Paso 1: Convertimos los resultados en DataFrame
df_validos = pd.DataFrame([r for r in resultados if "error" not in r])
df_errores = pd.DataFrame([r for r in resultados if "error" in r])

#  Paso 2: Aplanamos el campo "analisis"
if "analisis" in df_validos.columns:
    analisis_df = pd.json_normalize(df_validos["analisis"])
    df_validos = df_validos.drop(columns=["analisis"]).join(analisis_df)

#  Paso 3: Aplanamos el campo "comunicaciones"
if "comunicaciones" in df_validos.columns:
    comunicaciones_df = pd.json_normalize(df_validos["comunicaciones"])
    df_validos = df_validos.drop(columns=["comunicaciones"]).join(comunicaciones_df, rsuffix="_from_coms")

    # Renombramos las columnas para que se vean bien
    df_validos.rename(columns={
        "email_cliente_from_coms": "email_cliente",
        "email_cliente_traduccion_from_coms": "email_cliente_traduccion",
        "notificacion_interna_from_coms": "notificacion_interna",
        "email_proveedor_from_coms": "email_proveedor"
    }, inplace=True)

#  Paso 4: Mostramos una vista previa de las comunicaciones generadas
columnas_muestra = [
    "comentario_original", "idioma", "valoracion",
    "email_cliente", "email_cliente_traduccion",
    "notificacion_interna", "email_proveedor"
]

df_validos[columnas_muestra].head(5)


,comentario_original,idioma,valoracion,email_cliente,email_cliente_traduccion,notificacion_interna,email_proveedor
0,Comentario 1: Estoy muy contento con mis nueva...,español,negativa,¡Hola! Nos alegra mucho saber que estás conten...,,El cliente está muy contento con las zapatilla...,"Estimado proveedor, queremos felicitarle por l..."
1,Kommentar 2: Als die Schuhe angekommen sind ha...,alemán,negativa,"Lieber Kunde, vielen Dank für Ihr Feedback zu ...","Estimado cliente, gracias por tu opinión sobre...",El cliente ha mencionado problemas de durabili...,"Estimado proveedor, hemos recibido un comentar..."
2,Σχόλιο 3: Έλαβα τα παπούτσια μου σε λιγότερο α...,griego,negativa,Γεια σας! Χαίρομαι που είστε ικανοποιημένος με...,¡Hola! Me alegra que estés satisfecho con tu c...,El cliente ha expresado su satisfacción con la...,"Estimado proveedor, queremos informarte que el..."
3,Commentaire 4: La livraison a pris plus d'une ...,francés,negativa,"Cher client, Nous sommes désolés d'apprendre q...","Dear customer, We are sorry to hear that you h...",Se ha identificado que el cliente ha tenido pr...,"Estimado proveedor, Queremos informarle que un..."
4,Commento 5: Ho ricevuto le scarpe in meno di 4...,italiano,negativa,"Gentile cliente, grazie per il tuo feedback po...","Estimado cliente, gracias por tu comentario po...",El cliente ha dejado un comentario positivo so...,"Estimado proveedor, queremos informarte que un..."


#### 5.5 🛠️ *Aplicación de comunicaciones oficiales tras el análisis*

Una vez que todos los comentarios han sido procesados y normalizados, aplicamos esta función adicional para generar las comunicaciones definitivas.

Esta función añade al análisis de cada comentario:

- ✉️ `email_cliente`: texto de respuesta al cliente, en su idioma
- 📤 `notificacion_interna`: mensaje para el equipo de calidad o logística
- 🏭 `email_proveedor`: correo formal para el proveedor (si aplica)
- 🌐 `email_cliente_traduccion`: traducción al español si el comentario era en otro idioma

Este paso es imprescindible para que esas comunicaciones aparezcan luego en el Excel final.


In [ ]:
# Memoria de traducciones ya hechas
traducciones_cache = {}

# Aplicamos las comunicaciones oficiales a cada resultado válido
resultados_corregidos = []

for r in resultados:
    if "error" not in r:
        r = aplicar_comunicaciones_oficiales(r)
    resultados_corregidos.append(r)

# Sobrescribimos la lista de resultados con las comunicaciones ya añadidas
resultados = resultados_corregidos


🌍 Traduciendo a alemán...
🌍 Traduciendo a griego...
🌍 Traduciendo a francés...
🌍 Traduciendo a italiano...
🌍 Traduciendo a holandés...
🌍 Traduciendo a polaco...
🌍 Traduciendo a finlandés...
🌍 Traduciendo a portugués...
🌍 Traduciendo a inglés...
🌍 Traduciendo a francés...
🌍 Traduciendo a alemán...
🌍 Traduciendo a alemán...
🌍 Traduciendo a checo...
🌍 Traduciendo a holandés...
🌍 Traduciendo a húngaro...
🌍 Traduciendo a croata...
🌍 Traduciendo a inglés...
🌍 Traduciendo a finlandés...
🌍 Traduciendo a danés...
🌍 Traduciendo a búlgaro...
🌍 Traduciendo a checo...
🌍 Traduciendo a sueco...
🌍 Traduciendo a húngaro...
🌍 Traduciendo a rumano...
🌍 Traduciendo a polaco...
🌍 Traduciendo a italiano...
🌍 Traduciendo a sueco...
🌍 Traduciendo a portugués...
🌍 Traduciendo a griego...
🌍 Traduciendo a holandés...
🌍 Traduciendo a inglés...
🌍 Traduciendo a húngaro...
🌍 Traduciendo a croata...
🌍 Traduciendo a rumano...
🌍 Traduciendo a danés...
🌍 Traduciendo a búlgaro...


#### 5.6 👀 *Vista previa de traducciones automáticas del mensaje al cliente*

Comprobamos que el campo `email_cliente_traduccion` contiene una versión traducida automáticamente del mensaje generado en español, manteniendo las medidas de calidad y el tono profesional.


In [ ]:
# Mostramos una tabla con comentarios que no están en español para verificar la traducción automática
df_traducciones = df_validos[df_validos["idioma"] != "español"]

# Seleccionamos columnas relevantes
columnas_traduccion = [
    "comentario_original", "idioma", "email_cliente", "email_cliente_traduccion"
]

# Mostramos las primeras filas
display(df_traducciones[columnas_traduccion].head(5))


,comentario_original,idioma,email_cliente,email_cliente_traduccion
1,Kommentar 2: Als die Schuhe angekommen sind ha...,alemán,"Lieber Kunde, vielen Dank für Ihr Feedback zu ...","Estimado cliente, gracias por tu opinión sobre..."
2,Σχόλιο 3: Έλαβα τα παπούτσια μου σε λιγότερο α...,griego,Γεια σας! Χαίρομαι που είστε ικανοποιημένος με...,¡Hola! Me alegra que estés satisfecho con tu c...
3,Commentaire 4: La livraison a pris plus d'une ...,francés,"Cher client, Nous sommes désolés d'apprendre q...","Dear customer, We are sorry to hear that you h..."
4,Commento 5: Ho ricevuto le scarpe in meno di 4...,italiano,"Gentile cliente, grazie per il tuo feedback po...","Estimado cliente, gracias por tu comentario po..."
5,Opmerking 6: De schoenen arriveerden na meer d...,holandés,"Beste klant, We betreuren het ten zeerste dat ...","Estimado cliente, Lamentamos mucho que hayas r..."


## **6 Conversión de resultados a DataFrame y validación**

En este paso transformamos la lista `resultados` en dos `DataFrames` separados:

- `df_resultados`: contiene solo los comentarios que fueron analizados correctamente.
- `df_errores`: contiene los comentarios que generaron errores durante el análisis.

Esto nos permitirá seguir con la visualización y exportación solamente con los datos válidos, y revisar por separado los errores si los hubiera.



In [ ]:
# Separar los resultados válidos y los que contienen errores
resultados_validos = [r for r in resultados if "error" not in r]
resultados_con_errores = [r for r in resultados if "error" in r]

# Convertir a DataFrame
df_resultados = pd.json_normalize(resultados_validos)
df_errores = pd.json_normalize(resultados_con_errores)

# Mostrar resumen
print(f"✅ Comentarios procesados correctamente: {len(df_resultados)}")
print(f"⚠️ Comentarios con errores: {len(df_errores)}")

# Vista previa
print("\n🔍 Vista previa de resultados válidos:")
display(df_resultados.head(3))

if not df_errores.empty:
    print("\n🧪 Vista previa de errores:")
    display(df_errores[["comentario_original", "error"]].head(3))


✅ Comentarios procesados correctamente: 49
⚠️ Comentarios con errores: 1

🔍 Vista previa de resultados válidos:


,valoracion,comentario_original,email_cliente,email_cliente_traduccion,notificacion_interna,email_proveedor,analisis.idioma,analisis.envio_96h,analisis.embalaje_danado,analisis.talla_correcta,analisis.materiales_calidad,analisis.tipo_uso,analisis.cumple_expectativas,comunicaciones.email_cliente,comunicaciones.email_cliente_traduccion,comunicaciones.notificacion_interna,comunicaciones.email_proveedor
0,negativa,Comentario 1: Estoy muy contento con mis nueva...,Gracias por tu opinión. Estamos revisando los ...,,,,español,sí,no,sí,sí,diario,sí,¡Hola! Nos alegra mucho saber que estás conten...,,El cliente está muy contento con las zapatilla...,"Estimado proveedor, queremos felicitarle por l..."
1,negativa,Kommentar 2: Als die Schuhe angekommen sind ha...,Lamentamos mucho los inconvenientes encontrado...,"""Wir bedauern sehr die Unannehmlichkeiten, die...",Este cliente ha recibido una valoración negati...,Estimado proveedor:\n- Contactar con el provee...,alemán,no,no,sí,parcialmente,no mencionado,parcialmente,"Lieber Kunde, vielen Dank für Ihr Feedback zu ...","Estimado cliente, gracias por tu opinión sobre...",El cliente ha mencionado problemas de durabili...,"Estimado proveedor, hemos recibido un comentar..."
2,negativa,Σχόλιο 3: Έλαβα τα παπούτσια μου σε λιγότερο α...,Gracias por tu opinión. Estamos revisando los ...,"""Σας ευχαριστούμε για τη γνώμη σας. Επανεξετάζ...",,,griego,sí,no,sí,sí,diario,sí,Γεια σας! Χαίρομαι που είστε ικανοποιημένος με...,¡Hola! Me alegra que estés satisfecho con tu c...,El cliente ha expresado su satisfacción con la...,"Estimado proveedor, queremos informarte que el..."



🧪 Vista previa de errores:


,comentario_original,error
0,Commentaire 22: Les chaussures sont arrivées d...,❌ Respuesta vacía o malformada (no es JSON)


### 6.1 📂 *Separación de comentarios válidos y con errores + preparación para visualización*

Una vez procesados todos los comentarios por el asistente inteligente y aplicadas las verificaciones finales, dividimos los resultados en dos conjuntos:

- ✅ `df_validos`: comentarios correctamente analizados que contienen una valoración y comunicaciones generadas.
- ⚠️ `df_errores`: comentarios que no han podido ser procesados correctamente (por ejemplo, por errores de formato JSON o respuestas vacías del modelo).

Además, normalizamos las columnas `analisis` y `comunicaciones`, que vienen como diccionarios anidados, para que podamos visualizar cada variable como una columna independiente en los gráficos y tablas.

Este paso es clave para habilitar las siguientes visualizaciones interactivas y para exportar los resultados al Excel final.


In [ ]:
# ✅ Separa comentarios válidos de errores
df_validos = pd.DataFrame([r for r in resultados if "error" not in r])
df_errores = pd.DataFrame([r for r in resultados if "error" in r])

# ✅ Aplana el campo "analisis" si está presente
if "analisis" in df_validos.columns:
    analisis_df = pd.json_normalize(df_validos["analisis"])
    df_validos = df_validos.drop(columns=["analisis"]).join(analisis_df)

# ✅ Aplana el campo "comunicaciones" si está presente, evitando solapamientos
if "comunicaciones" in df_validos.columns:
    comunicaciones_df = pd.json_normalize(df_validos["comunicaciones"])
    df_validos = df_validos.drop(columns=["comunicaciones"]).join(comunicaciones_df, rsuffix="_from_coms")

    # Renombramos para que el Excel tenga los nombres esperados
    df_validos.rename(columns={
        "email_cliente_from_coms": "email_cliente",
        "email_cliente_traduccion_from_coms": "email_cliente_traduccion",
        "notificacion_interna_from_coms": "notificacion_interna",
        "email_proveedor_from_coms": "email_proveedor"
    }, inplace=True)



In [ ]:
# 🔍 Vista previa de columnas clave con las comunicaciones generadas
columnas_muestra = [
    "comentario_original", "email_cliente", "notificacion_interna", "email_proveedor"
]

# Mostramos las primeras filas con las columnas de interés
df_validos[columnas_muestra].head(3)


,comentario_original,email_cliente,email_cliente,notificacion_interna,notificacion_interna,email_proveedor,email_proveedor
0,Comentario 1: Estoy muy contento con mis nueva...,Gracias por tu opinión. Estamos revisando los ...,¡Hola! Nos alegra mucho saber que estás conten...,,El cliente está muy contento con las zapatilla...,,"Estimado proveedor, queremos felicitarle por l..."
1,Kommentar 2: Als die Schuhe angekommen sind ha...,Lamentamos mucho los inconvenientes encontrado...,"Lieber Kunde, vielen Dank für Ihr Feedback zu ...",Este cliente ha recibido una valoración negati...,El cliente ha mencionado problemas de durabili...,Estimado proveedor:\n- Contactar con el provee...,"Estimado proveedor, hemos recibido un comentar..."
2,Σχόλιο 3: Έλαβα τα παπούτσια μου σε λιγότερο α...,Gracias por tu opinión. Estamos revisando los ...,Γεια σας! Χαίρομαι που είστε ικανοποιημένος με...,,El cliente ha expresado su satisfacción con la...,,"Estimado proveedor, queremos informarte que el..."


## 📊 **7.  Resultados analíticos del sistema IA del análisis de comentarios**
En esta sección presentamos un resumen visual interactivo del análisis de los comentarios de clientes realizado por el asistente inteligente de KelceTS S.L.

Gracias a la integración con modelos de lenguaje y las reglas definidas por la empresa, se ha procesado automáticamente cada comentario multilingüe para extraer:
- La valoración general,
- El idioma,
- Las incidencias relacionadas con envío, talla o materiales,
- Y las respuestas personalizadas generadas para cada cliente.

A continuación se muestran los resultados visualizados mediante gráficos interactivos y tablas dinámicas para facilitar el análisis final. Esta sección también sirve como informe visual para la entrega del proyecto.


### 7.1 📊 Distribución de valoraciones por idioma

Este gráfico muestra cómo se distribuyen las valoraciones (positiva, negativa, parcialmente positiva) en función del idioma del comentario original. Permite detectar tendencias por idioma o regiones, y ayuda a identificar patrones de satisfacción o insatisfacción en distintos mercados europeos.


In [ ]:
# Asegurarse de tener columnas correctas
if not df_validos.empty and "idioma" in df_validos.columns and "valoracion" in df_validos.columns:
    fig = px.histogram(
        df_validos,
        x="idioma",
        color="valoracion",
        barmode="group",
        title="Distribución de valoraciones por idioma",
        labels={"idioma": "Idioma del comentario", "valoracion": "Valoración"},
        category_orders={"idioma": sorted(df_validos["idioma"].unique())},
        color_discrete_sequence=px.colors.qualitative.Pastel
    )
    fig.update_layout(
        xaxis_title="Idioma",
        yaxis_title="Número de comentarios",
        legend_title="Valoración",
        title_font_size=20
    )
    fig.write_image("grafico_valoraciones_idioma.png")  # Para exportarlo al Excel luego
    fig.show()
else:
    print("⚠️ No hay datos cargados para mostrar el gráfico.")


### 7.2 🧭 Distribución de valoraciones generales (positiva, negativa, parcialmente)

Este gráfico de pastel muestra de forma visual el desglose total de las valoraciones emitidas por el asistente de IA tras analizar cada comentario.

Cada porción representa el porcentaje de comentarios que han sido evaluados como:

- ✅ Positivos
- ⚠️ Parcialmente positivos
- ❌ Negativos

Este gráfico nos permite tener una visión general del nivel de satisfacción global de los clientes en base a sus comentarios.


In [ ]:
# Asegurarse de que hay datos válidos
if not df_validos.empty and "valoracion" in df_validos.columns:
    valoraciones_counts = df_validos["valoracion"].value_counts().reset_index()
    valoraciones_counts.columns = ["Valoración", "Cantidad"]

    fig_val = px.pie(
        valoraciones_counts,
        names="Valoración",
        values="Cantidad",
        title="🧭 Distribución de valoraciones generales",
        color_discrete_sequence=px.colors.qualitative.Safe
    )
    fig_val.update_traces(textposition="inside", textinfo="percent+label")
    fig_val.write_image("grafico_valoraciones_pastel.png")  # Exportamos como imagen
    fig_val.show()
else:
    print("⚠️ No hay datos cargados para generar el gráfico de valoraciones.")


### 7.3 ⚠️ Distribución de errores por idioma estimado

Este gráfico muestra cuántos comentarios no pudieron ser procesados correctamente por el asistente de IA, agrupados por el idioma estimado a partir del texto original.

Los errores pueden deberse a:
- Respuestas vacías o malformadas por parte del modelo,
- Comentarios con estructuras poco comunes o en idiomas complejos,
- Problemas de codificación o truncamiento en la entrada.

Este análisis permite detectar en qué idiomas o mercados el modelo tiene más dificultades, lo cual es clave para mejorar su rendimiento futuro.

> ⚠️ Nota: El idioma ha sido estimado a partir de palabras clave del comentario original, ya que estos casos no pudieron ser analizados completamente.


In [ ]:
# Estimamos el idioma de cada comentario con error en función del encabezado
if not df_errores.empty and "comentario_original" in df_errores.columns:
    idioma_respaldo = []
    for comentario in df_errores["comentario_original"]:
        if "Comentario" in comentario:
            idioma_respaldo.append("español")
        elif "Kommentar" in comentario:
            idioma_respaldo.append("alemán")
        elif "Σχόλιο" in comentario:
            idioma_respaldo.append("griego")
        elif "Kommentti" in comentario:
            idioma_respaldo.append("finés")
        elif "Commentaire" in comentario:
            idioma_respaldo.append("francés")
        elif "Kommentér" in comentario:
            idioma_respaldo.append("danés")
        elif "Komment" in comentario:
            idioma_respaldo.append("húngaro")
        else:
            idioma_respaldo.append("desconocido")

    df_errores["idioma_estimado"] = idioma_respaldo

    # Contamos errores por idioma
    errores_por_idioma = df_errores["idioma_estimado"].value_counts().reset_index()
    errores_por_idioma.columns = ["Idioma", "Errores"]

    # Generamos gráfico
    fig_errores = px.bar(
        errores_por_idioma,
        x="Idioma",
        y="Errores",
        title="⚠️ Distribución de errores por idioma estimado",
        color="Errores",
        color_continuous_scale="reds"
    )
    fig_errores.update_layout(
        xaxis_title="Idioma estimado",
        yaxis_title="Número de errores",
        title_font_size=20
    )
    fig_errores.write_image("grafico_errores_idioma.png")  # Se guarda para exportar al Excel
    fig_errores.show()
else:
    print("✅ No hay errores detectados. ¡Todo ha sido procesado correctamente!")


### 7.4 🔍 Análisis de variables clave de calidad

Este gráfico muestra el desglose de las respuestas detectadas por el asistente para cada una de las categorías de calidad evaluadas en los comentarios:

- 📦 Entrega en menos de 96 horas (`envio_96h`)
- 📉 Embalaje en buen estado (`embalaje_danado`)
- 👟 Talla correcta (`talla_correcta`)
- 🧵 Calidad de los materiales (`materiales_calidad`)
- 🌟 Cumplimiento de expectativas del cliente (`cumple_expectativas`)

Cada categoría refleja cuántos comentarios han indicado que **se cumple** (`sí`), que **no se cumple** (`no`) o que la información es **parcial** o dudosa (`parcialmente`). Esto ayuda a identificar las áreas de mejora más relevantes según el feedback de los usuarios.


In [ ]:
# Definimos las variables clave que queremos analizar
variables_calidad = ["envio_96h", "embalaje_danado", "talla_correcta", "materiales_calidad", "cumple_expectativas"]

# Preparar una lista con los datos de conteo para cada variable
datos_calidad = []
for var in variables_calidad:
    if var in df_validos.columns:
        counts = df_validos[var].value_counts().to_dict()
        for valor, cantidad in counts.items():
            datos_calidad.append({"Categoría": var, "Valor": valor, "Cantidad": cantidad})

# Convertimos a DataFrame
df_calidad = pd.DataFrame(datos_calidad)

# Creamos el gráfico de barras agrupadas
fig_calidad = px.bar(
    df_calidad,
    x="Categoría",
    y="Cantidad",
    color="Valor",
    barmode="group",
    title="🔍 Análisis de variables clave de calidad",
    color_discrete_sequence=px.colors.qualitative.Set2
)
fig_calidad.update_layout(
    xaxis_title="Categoría evaluada",
    yaxis_title="Número de comentarios",
    legend_title="Respuesta"
)

# Guardamos la imagen para exportarla al Excel
fig_calidad.write_image("grafico_calidad_por_variable.png")
fig_calidad.show()

# 8. 📤 **Exportación final con comunicaciones correctas**

En este bloque se realiza lo siguiente:

1. Se recorre cada resultado y se aplica la función `aplicar_comunicaciones_oficiales(resultado)`, la cual genera los textos oficiales para:
   - Email al cliente,
   - Notificación interna,
   - Email al proveedor,  
   según las reglas definidas en el prompt.

2. Se agrupan estas comunicaciones dentro de la clave `comunicaciones` para facilitar su exportación.

3. Se crean los DataFrames para los resultados válidos y los que tienen error, se aplanan las estructuras anidadas y se renombran las columnas de comunicaciones (usando un sufijo temporal para evitar solapamientos).

4. Finalmente, se exporta a un Excel con tres hojas:  
   - "Comentarios válidos" (con las comunicaciones completas),
   - "Errores detectados",
   - "Resumen visual" (donde se insertan imágenes de gráficos ya generados).

Este paso garantiza que el Excel final refleje exactamente lo definido en las reglas del ejercicio práctico.



In [ ]:
def aplicar_comunicaciones_oficiales(resultado):
    idioma = resultado.get("idioma", "español")
    uso = resultado.get("tipo_uso", "")
    valoracion = resultado.get("valoracion", "")

    # Traducción solo si idioma no es español
    if idioma != "español":
        resultado["email_cliente_traduccion"] = "Traducción automática disponible bajo solicitud."
    else:
        resultado["email_cliente_traduccion"] = ""

    # Firma y email final
    firma = "\n\nUn cordial saludo,\nKelceTS Team\natencionalcliente@kelcetssl.com"

    # Inicializamos mensajes
    comunicacion_cliente = ""
    comunicacion_interna = ""
    comunicacion_proveedor = ""

    if valoracion == "positiva":
        comunicacion_cliente = f"¡Gracias por confiar en KelceTS! Nos alegra saber que estás satisfecho con tu compra.{firma}"

    elif valoracion == "negativa":
        piezas = []

        if resultado.get("envio_96h") == "no":
            piezas.append("Hemos detectado un retraso en la entrega. Te ofrecemos un 5% de descuento en tu próxima compra.")
            comunicacion_proveedor += "- Contactar con el proveedor para mejorar el tiempo de entrega (menos de 24h).\n"

        if resultado.get("embalaje_danado") == "sí":
            piezas.append("Hemos registrado que el embalaje llegó dañado. Te ofrecemos un 5% de descuento por las molestias.")
            comunicacion_proveedor += "- Evaluar calidad del embalaje con proveedor logístico.\n"

        if resultado.get("talla_correcta") == "no":
            piezas.append("Vamos a enviarte en menos de 72h un nuevo par con la talla correcta, sin coste. Por favor, prepara el par anterior para su recogida.")
            comunicacion_proveedor += "- Envío urgente de nuevo par con talla correcta. Recojo del anterior.\n"

        if resultado.get("materiales_calidad") == "no":
            piezas.append("Te ofrecemos un 25% de descuento y envío gratis. Enviaremos un miembro del staff en 72h para recoger el producto defectuoso.")
            comunicacion_proveedor += "- Revisar materiales con proveedor. Plan de sustitución urgente.\n"

        if piezas:
            comunicacion_cliente = "Lamentamos mucho los inconvenientes encontrados en tu compra. " + " ".join(piezas) + firma
            comunicacion_interna = "Este cliente ha recibido una valoración negativa. Deben notificarse los siguientes puntos:\n" + comunicacion_proveedor + "\nFirmado: Asistente IA de KelceTS S.L."
            comunicacion_proveedor = "Estimado proveedor:\n" + comunicacion_proveedor + "\nAtentamente,\nRodrigo Clemente, Director de Logística de KelceTS S.L."

    elif valoracion == "parcialmente":
        comunicacion_cliente = f"Gracias por tu opinión. Vamos a revisar los aspectos que mencionas para seguir mejorando.{firma}"

    # Asignamos campos generados
    resultado["email_cliente"] = comunicacion_cliente
    resultado["notificacion_interna"] = comunicacion_interna
    resultado["email_proveedor"] = comunicacion_proveedor

    return resultado


#### 8.1✅ *Validación final de las comunicaciones generadas*

Antes de entregar el archivo Excel, realizamos una validación automática de las comunicaciones generadas por el asistente.

Esta revisión asegura que:

- Se aplican las medidas de calidad correctas (5%, 25%, envío nuevo par...)
- Las firmas de los correos son las oficiales (KelceTS Team, Rodrigo Clemente)
- No hay respuestas vacías o incompletas

Este control actúa como una auditoría automática del contenido generado.


In [ ]:
# Validación de contenidos en comunicaciones

def check_keywords(text, keywords):
    if not isinstance(text, str):
        return False
    return any(k.lower() in text.lower() for k in keywords)

# Palabras clave que esperamos encontrar
requisitos = {
    "email_cliente": ["gracias por confiar", "lamentamos", "nuevo par", "25%", "5%"],
    "email_proveedor": ["Rodrigo Clemente", "logística", "proveedor", "evaluar"],
    "notificacion_interna": ["cliente ha recibido", "firmado", "sustitución", "notificar"]
}

# Creamos reporte de validación
errores_validacion = []

for i, row in df_validos.iterrows():
    for campo, claves in requisitos.items():
        texto = row.get(campo, "")
        if not check_keywords(texto, claves):
            errores_validacion.append({
                "comentario_original": row.get("comentario_original", ""),
                "campo_fallido": campo,
                "texto_actual": texto
            })

df_errores_validacion = pd.DataFrame(errores_validacion)

if df_errores_validacion.empty:
    print("✅ Todas las comunicaciones cumplen con las reglas oficiales.")
else:
    print(f"⚠️ Se encontraron {len(df_errores_validacion)} comunicaciones que podrían no cumplir las reglas.")
    display(df_errores_validacion.head())


⚠️ Se encontraron 147 comunicaciones que podrían no cumplir las reglas.


,comentario_original,campo_fallido,texto_actual
0,Comentario 1: Estoy muy contento con mis nueva...,email_cliente,email_cliente Gracias por tu opinión. Estam...
1,Comentario 1: Estoy muy contento con mis nueva...,email_proveedor,email_proveedor ...
2,Comentario 1: Estoy muy contento con mis nueva...,notificacion_interna,notificacion_interna ...
3,Kommentar 2: Als die Schuhe angekommen sind ha...,email_cliente,email_cliente Lamentamos mucho los inconven...
4,Kommentar 2: Als die Schuhe angekommen sind ha...,email_proveedor,email_proveedor Estimado proveedor:\n- Cont...


#### 8.2 🔎 *Verificamos que las comunicaciones generadas están presentes*

Mostramos una muestra de los textos generados para:
- `email_cliente`
- `notificacion_interna`
- `email_proveedor`

Si aparecen correctamente aquí, el problema está en el momento de construir el Excel. Si están vacíos, hay que revisar su generación.


In [ ]:
# Mostramos una muestra de resultados con comunicaciones
print("🔍 Mostrando muestra de comunicaciones generadas:\n")

ejemplos_mostrar = 5

for i, r in enumerate(resultados):
    if "error" not in r and "comunicaciones" in r:
        print(f"🧾 Comentario {i+1}:")
        print("✉️ Email Cliente:\n", r["comunicaciones"].get("email_cliente", "⚠️ VACÍO"))
        print("📤 Notificación Interna:\n", r["comunicaciones"].get("notificacion_interna", "⚠️ VACÍA"))
        print("🏭 Email Proveedor:\n", r["comunicaciones"].get("email_proveedor", "⚠️ VACÍO"))
        print("\n" + "-"*80 + "\n")
        ejemplos_mostrar -= 1
        if ejemplos_mostrar == 0:
            break


🔍 Mostrando muestra de comunicaciones generadas:

🧾 Comentario 1:
✉️ Email Cliente:
 ¡Hola! Nos alegra mucho saber que estás contento con tus nuevas zapatillas. Gracias por compartir tu experiencia positiva con nosotros. Si en el futuro necesitas algo más, no dudes en contactarnos. ¡Que disfrutes al correr con ellas! KelceTS Team
📤 Notificación Interna:
 El cliente está muy contento con las zapatillas recibidas, destacando la rapidez en la entrega, el buen embalaje, la talla correcta y la calidad de los materiales. Utiliza las zapatillas a diario y afirma que han superado sus expectativas.
🏭 Email Proveedor:
 Estimado proveedor, queremos felicitarle por la excelente calidad de las zapatillas suministradas al cliente. Hemos recibido un comentario muy positivo destacando la rapidez en la entrega, el buen embalaje, la talla correcta y la calidad de los materiales. ¡Gracias por su buen trabajo! Rol: Equipo de Calidad

------------------------------------------------------------------------

#### 8.3 📢 *Exportación Final de Resultados del Análisis*

En esta etapa, se ha consolidado todo el trabajo realizado en el análisis automatizado de los comentarios de clientes. El archivo Excel generado, `Informe_Final_KelceTS.xlsx`, integra toda la información relevante y verificada para su revisión final.

Esta celda genera el archivo  con:

1. 🟩 Comentarios válidos (con todas las comunicaciones generadas)
2. 🟨 Comentarios con error (si los hay)
3. 📊 Gráficos de resumen visual en una tercera hoja

El archivo se descargará automáticamente al finalizar.

Este documento es la consolidación de todo el proceso.


In [ ]:

from google.colab import files

# Ruta del archivo final
ruta_excel = "Informe_Final_KelceTS.xlsx"

# Columnas seleccionadas para la hoja principal
columnas_entrega = [
    "comentario_original", "idioma", "valoracion",
    "envio_96h", "embalaje_danado", "talla_correcta", "materiales_calidad",
    "tipo_uso", "cumple_expectativas",
    "email_cliente", "email_cliente_traduccion",
    "notificacion_interna", "email_proveedor"
]

# Filtramos solo las columnas disponibles
columnas_entrega = [col for col in columnas_entrega if col in df_validos.columns]
df_validos_entrega = df_validos[columnas_entrega]

# Exportamos al Excel
with pd.ExcelWriter(ruta_excel, engine="xlsxwriter") as writer:
    # Hoja 1: Comentarios válidos
    df_validos_entrega.to_excel(writer, sheet_name="Comentarios válidos", index=False)

    # Hoja 2: Comentarios con errores
    if not df_errores.empty:
        df_errores.to_excel(writer, sheet_name="Errores detectados", index=False)

    # Hoja 3: Resumen visual (gráficos si están generados)
    workbook = writer.book
    worksheet = workbook.add_worksheet("Resumen visual")
    writer.sheets["Resumen visual"] = worksheet

    imagenes = [
        ("grafico_valoraciones_idioma.png", "Gráfico: Valoraciones por Idioma", "B2"),
        ("grafico_valoraciones_pastel.png", "Gráfico: Valoraciones Generales", "B22"),
        ("grafico_errores_idioma.png", "Gráfico: Errores por Idioma", "B42"),
        ("grafico_calidad_por_variable.png", "Gráfico: Variables de Calidad", "B62"),
    ]

    for archivo, titulo, celda in imagenes:
        try:
            worksheet.write(celda.replace("2", "1"), titulo)
            worksheet.insert_image(celda, archivo, {"x_scale": 0.8, "y_scale": 0.8})
        except Exception as e:
            print(f"⚠️ No se pudo insertar {archivo}: {e}")

# Descarga automática
print("🎉 ¡Informe generado con éxito!")
print("📁 Descargando archivo: Informe_Final_KelceTS.xlsx...")
files.download(ruta_excel)

🎉 ¡Informe generado con éxito!
📁 Descargando archivo: Informe_Final_KelceTS.xlsx...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

##### *8.3.1 📄 Exportación final a Excel y sincronización con Streamlit*
En este paso, exportamos el dataframe final df_resultado_final generado por el análisis automático de los comentarios con IA y reglas internas a un archivo Excel llamado: Informe_Final_KelceTS.xlsx

Este archivo se guarda en la carpeta data/, lo que permite:

1. ✅ Mantener una estructura coherente con el dashboard profesional desarrollado en Streamlit (como se puede ejecutar con el entregable 3 de este proyecto completo).
2. ✅ Compartir la misma base de datos final enriquecida entre el notebook y la app web.
3. ✅ Descargar automáticamente el archivo para su entrega o revisión.

>⚠️ Aunque técnicamente sería posible automatizar la subida directa a GitHub desde Colab usando un token de acceso personal (PAT), hemos optado por realizar esta subida de forma manual para:



*   Evitar exponer credenciales en el código
*   Cumplir con las buenas prácticas de seguridad
*   Facilitar la transparencia y trazabilidad para el profesorado



>🔁 Por tanto, este archivo se ha subido manualmente a la carpeta /data de mi repositorio GitHub, desde donde el dashboard de Streamlit lo utiliza como fuente de análisis visual.

# 📘 **9. Conclusiones y Trabajo Futuro**

Este ejercicio práctico demuestra cómo aplicar la Inteligencia Artificial Generativa para automatizar la gestión de comentarios multilingües en una startup ficticia de zapatillas online: **KelceTS S.L.**

A partir de un archivo `.txt` con 50 comentarios reales en 24 idiomas oficiales de la UE, se ha desarrollado un flujo completo que:

- 🧠 Analiza automáticamente los comentarios e identifica variables clave: envío, embalaje, talla, calidad, uso y expectativas
- 🗂 Clasifica cada comentario como positivo, negativo o parcialmente positivo
- ✉️ Genera correos personalizados al cliente en su idioma original, incluyendo medidas de calidad (descuentos, envíos urgentes, recogidas...)
- 📤 Informa internamente a los equipos de calidad y logística si corresponde
- 🏭 Redacta emails al proveedor externo si hay errores atribuibles a la cadena de suministro
- 🌍 Traduce automáticamente las respuestas generadas en castellano al idioma original del cliente, utilizando OpenAI y Gemini como fallback
- 📊 Exporta todos los resultados, respuestas y gráficas a un archivo Excel profesional con tres hojas: comentarios válidos, errores y resumen visual

---

## 🚀 Posibles mejoras futuras

- Incluir una capa de análisis de tono y emociones para adaptar el estilo de respuesta
- Añadir soporte multimodal (imágenes de productos dañados o vídeos de queja)
- Integrar feedback humano directo para mejorar la evaluación automática
- Conectarlo directamente a canales reales (email, WhatsApp, chatbot) para implementación real
- Desplegar como servicio API para integración con CRMs u otras plataformas de soporte

---

# **10. 📘 Agradecimientos y Cierre**

Gracias al equipo docente del Institututo de Inteligencia Artificial.
Ha sido toda una experiencia súper enriquecedoara formarme con vosotros.

A todos los que me puedan leer os recomiendo formaros con ellos. Os dejo aqúi su link: https://iia.es/

¡Muchísimas gracias! 😍

**Araceli Fradejas Muñoz**
